In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [4]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
class ConvBlock(tf.keras.Model):
    def __init__(self,channel):
        super(ConvBlock,self).__init__()
        
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(channel*4, kernel_size=1, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(channel, kernel_size=3, padding='same')
        self.concate = tf.keras.layers.Concatenate(axis=-1)
        
    def call(self, input_tensor, training = False):
        x = self.bn1(input_tensor, training=training)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.concate([input_tensor, x])
        return x

In [6]:
class DenseBlock(tf.keras.Model):
    def __init__(self, growth_rate ,block_nums):
        super(DenseBlock,self).__init__()
        self.model = tf.keras.Sequential()
        
        for _ in range(block_nums):
            self.model.add(ConvBlock(growth_rate))
        
        
    def call(self, input_tensor, training = False):
        x = self.model(input_tensor)
        return x

In [7]:
class TransitionBlock(tf.keras.Model):
    def __init__(self, channel):
        super(TransitionBlock,self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(channel,kernel_size=1)
        self.avgpool = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2)
        
    def call(self, input_tensor, training = False):
        x = self.bn1(input_tensor)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        x = self.avgpool(x) 
        return x

In [8]:
class DenseNet121(tf.keras.Model):
    def __init__(self, growth_rate=32):
        super(DenseNet121,self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(64, kernel_size = 7, strides=2, padding='same', use_bias=False)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.maxpool = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
        
        channel = 64
        
        self.dense_block1 = DenseBlock(growth_rate,block_nums = 6)
        channel = int((channel + growth_rate * 6)*0.5)
        self.transition_block1 = TransitionBlock(channel)
        self.dense_block2 = DenseBlock(growth_rate,block_nums = 12)
        channel = int((channel + growth_rate * 12)*0.5)
        self.transition_block2 = TransitionBlock(channel)
        self.dense_block3 = DenseBlock(growth_rate,block_nums = 32)
        channel = int((channel + growth_rate * 32)*0.5)
        self.transition_block3 = TransitionBlock(channel)
        self.dense_block4 = DenseBlock(growth_rate,block_nums = 32)

        
        self.bn2 = tf.keras.layers.BatchNormalization()       

        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc3 = tf.keras.layers.Dense(10, activation='softmax')        
        
    def call(self, input_tensor,training = False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.maxpool(x)
        
        x = self.dense_block1(x)
        x = self.transition_block1(x)
        x = self.dense_block2(x)
        x = self.transition_block2(x)
        x = self.dense_block3(x)
        x = self.transition_block3(x)
        x = self.dense_block4(x)
        
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
            
        
        x = self.gap(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [9]:
BATCH_SIZE = 256
EPOCH = 20

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [10]:
densenet121 = DenseNet121()

densenet121.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_densenet121 = densenet121.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1
)

Epoch 1/20
195/195 [==============================] - 24s 124ms/step - loss: 1.9768 - accuracy: 0.2844 - val_loss: 2.4194 - val_accuracy: 0.0978
Epoch 2/20
195/195 [==============================] - 20s 103ms/step - loss: 1.5941 - accuracy: 0.4121 - val_loss: 2.5362 - val_accuracy: 0.1426
Epoch 3/20
195/195 [==============================] - 20s 104ms/step - loss: 1.4350 - accuracy: 0.4732 - val_loss: 1.6154 - val_accuracy: 0.4081
Epoch 4/20
195/195 [==============================] - 20s 104ms/step - loss: 1.3208 - accuracy: 0.5195 - val_loss: 1.4716 - val_accuracy: 0.4662
Epoch 5/20
195/195 [==============================] - 20s 103ms/step - loss: 1.2169 - accuracy: 0.5610 - val_loss: 1.4040 - val_accuracy: 0.4904
Epoch 6/20
195/195 [==============================] - 20s 103ms/step - loss: 1.1261 - accuracy: 0.5969 - val_loss: 1.3599 - val_accuracy: 0.5132
Epoch 7/20
195/195 [==============================] - 20s 104ms/step - loss: 1.0271 - accuracy: 0.6320 - val_loss: 1.4105 - val_ac